In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Convert the dataset
with open('data/nat2021.csv') as f:
    data = f.read().splitlines()[1:]

data = [x.split(';')[1] for x in data]

# Remove data starting with _
data = [x for x in data if not x.startswith('_')]

# Remove one letter words
data = [x for x in data if len(x) > 1]

# Remove duplicates
data = list(set(data))

# Sort the list
data.sort()
data[:8]

In [ ]:
len(data)

In [ ]:
chars = sorted(list(set(''.join(data))))
stoi = {ch: i+1 for i, ch in enumerate(chars)}
stoi['.'] = 0
itos = {i: ch for ch, i in stoi.items()}
print(itos)

In [100]:
block_size = 3
X, Y = [], []
for word in data:
    context = [0] * block_size
    for ch in word + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [101]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([274508, 3]), torch.int64, torch.Size([274508]), torch.int64)

In [102]:
g = torch.Generator().manual_seed(42)
C = torch.randn((len(stoi), 2), generator=g)
W1 = torch.rand((6, 100), generator=g)
b1 = torch.rand(100, generator=g)
W2 = torch.rand((100, len(stoi)), generator=g)
b2 = torch.rand(len(stoi), generator=g)
parameters = [C, W1, b1, W2, b2]

In [103]:
sum(p.nelement() for p in parameters)

5438

In [104]:
for p in parameters:
    p.requires_grad = True

In [109]:
for i in range(100):
    # mini-batch of 1000
    ix = torch.randint(0, X.shape[0], (32,))


    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])

    for p in parameters:
        p.grad = None
    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad
print(loss.item())

2.715507984161377


In [ ]:
logits.max(1)